<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
LIGHT TABLE - GENERATION</p><br>

This notebook generate Light Tables based on nexus full-simulations of light, generating a DataFrame that should be used by DetSim.

TAREAS POR IMPLEMENTAR:

* Descripcion de lo que hace cada una de las funciones y sus parametros
* Anyadir Tablas de Configuracion a la salida
* Leer el setup de un fichero de configuracion
* Anyadir tests
* Posibilidad de muestrear los volumenes / areas con un patron asimetrico (mas denso a radios altos, para reflejar mejor los cambios que se producen cerca de los bordes).
* Hacer el setting de IC y de NEXUS desde las funciones, no desde el terminal
* Anyadir tests de que las simulaciones han corrido bien en las colas.

In [1]:
from IPython.core.display import HTML
css = open('css/style-table.css').read() + open('css/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [2]:
# General importings
import os
import numpy  as np
import pandas as pd

from typing import Tuple
from math   import ceil

# Specific IC stuff
import invisible_cities.core.system_of_units  as units
from invisible_cities.io.mcinfo_io        import load_mcsensor_response_df
from invisible_cities.io.mcinfo_io        import get_sensor_types

# Light Table stuff
from sim_functions       import make_init_file
from sim_functions       import make_config_file
from sim_functions       import run_sim
from sim_functions       import get_num_photons

from table_functions     import get_table_positions
from table_functions     import get_working_paths
from table_functions     import build_table
from table_functions     import get_fnames
from table_functions     import get_table_fname


In [3]:
%load_ext autoreload
%autoreload 2

# SETUP DATA

In [4]:
VERBOSITY = True

In [7]:
MAX_PHOTONS_PER_EVT = 100000

In [8]:
### Current options: "NEXT_NEW", "NEXT100", "NEXT_FLEX"
det_name = "NEXT_NEW"

### Type of Light Table: energy or tracking
table_type = "tracking"

### Signal Type: S1 or S2
signal_type = "S2"


### Sensor name.
# Tipically PmtR11410 for energy tables and SiPM for tracking tables. 
#sensor_name = "PmtR11410"
sensor_name = "SiPM"

### Table pitch
#pitch = (200.0 * units.mm, 200.0 * units.mm, 200.0 * units.mm)
#pitch = (20.0 * units.mm, 20.0 * units.mm, 40.0 * units.mm)
pitch = (30.0 * units.mm, 30.0 * units.mm, 1.0 * units.mm)
#pitch = (1.0 * units.mm, 1.0 * units.mm, 1.0 * units.mm)

### Table num photons / point
photons_per_point = 100000


### Verbosity
if VERBOSITY:
    print(f"***** Generating {det_name} Light Table  *****\n")
    print(f"*** Type: {table_type}  -  Signal: {signal_type}  -  Sensor: {sensor_name}")
    print(f"*** Pitch: {pitch} mm - Photons/Point = {photons_per_point:.1e}")

***** Generating NEXT_NEW Light Table  *****

*** Type: tracking  -  Signal: S2  -  Sensor: SiPM
*** Pitch: (30.0, 30.0, 1.0) mm - Photons/Point = 1.0e+05


In [9]:
# Getting (num_evts / file) & num_evts
num_evts          = 1
photons_per_event = photons_per_point

if photons_per_event > MAX_PHOTONS_PER_EVT:
    num_evts          = ceil(photons_per_point / MAX_PHOTONS_PER_EVT)
    photons_per_event = MAX_PHOTONS_PER_EVT
    photons_per_point = num_evts * photons_per_event

    if VERBOSITY:
        print(f"*** {num_evts} Events/Point of {photons_per_event:.1e} photons ...")
        print(f"    -> {photons_per_point:.1e} Photons/Point")

else:
    if VERBOSITY:
        print(f"*** Photons/Event: {photons_per_event:.1e}")

*** 10 Events/Point of 1.0e+04 photons ...
    -> 1.0e+05 Photons/Point


In [10]:
table_positions = get_table_positions(det_name, table_type, signal_type, pitch)

# Vervosity
if VERBOSITY:
    print(f"*** {len(table_positions)} points ...")
    print(table_positions)

*** 18 points ...
[(25.0, 25.0, -0.5), (25.0, 25.0, -1.5), (25.0, 25.0, -2.5), (25.0, 25.0, -3.5), (25.0, 25.0, -4.5), (25.0, 25.0, -5.5), (55.0, 25.0, -0.5), (55.0, 25.0, -1.5), (55.0, 25.0, -2.5), (55.0, 25.0, -3.5), (55.0, 25.0, -4.5), (55.0, 25.0, -5.5), (85.0, 25.0, -0.5), (85.0, 25.0, -1.5), (85.0, 25.0, -2.5), (85.0, 25.0, -3.5), (85.0, 25.0, -4.5), (85.0, 25.0, -5.5)]


In [11]:
# Getting PATHS
config_path, log_path, dst_path, table_path = get_working_paths(det_name)

# Verbosity
if VERBOSITY:
    print(f"* Config PATH: {config_path}")
    print(f"* Log    PATH: {log_path}")
    print(f"* Dst    PATH: {dst_path}")
    print(f"* Table  PATH: {table_path}")

* Config PATH: /Users/Javi/Development/NextLightTable/data/NEXT_NEW/config/
* Log    PATH: /Users/Javi/Development/NextLightTable/data/NEXT_NEW/log/
* Dst    PATH: /Users/Javi/Development/NextLightTable/data/NEXT_NEW/dst/
* Table  PATH: /Users/Javi/Development/NextLightTable/data/NEXT_NEW/table/


# LIGHT SIMULATIONS

In [12]:
for pos in table_positions:
    
    # file names
    init_fname, config_fname, log_fname, dst_fname = get_fnames(det_name ,pos)

    # make configuration files
    make_init_file(det_name, init_fname, config_fname)
    
    make_config_file(det_name, config_fname, dst_fname,
                     pos[0], pos[1], pos[2],
                     photons_per_event)
    
    # Runing the simulation
    if VERBOSITY:
        print(f"* Runing {det_name} sim of {photons_per_point:.1e} photons from {pos} ...")
        
    # Check if the sim is already run with the correct num_photons
    if os.path.isfile(dst_fname + '.h5'):
        run_photons = get_num_photons(dst_fname + '.h5')
        if (get_num_photons(dst_fname + '.h5') < photons_per_point):
            print("  Simulation already run previously with less events, so re-running ...")
            run_sim(init_fname, log_fname, num_evts)
        else:
            print("  Simulation already run previously, so skipping ...")
    
    else:
        run_sim(init_fname, log_fname, num_evts)

* Runing NEXT_NEW sim of 1.0e+05 photons from (25.0, 25.0, -0.5) ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (25.0, 25.0, -1.5) ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (25.0, 25.0, -2.5) ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (25.0, 25.0, -3.5) ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (25.0, 25.0, -4.5) ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (25.0, 25.0, -5.5) ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (55.0, 25.0, -0.5) ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (55.0, 25.0, -1.5) ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (55.0, 25.0, -2.5) ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (55.0, 25.0, -3.5) ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (55.0, 25.0, -4.5) ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (55.0, 25.0, -5.5) ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (85.0, 25.0, -0.5) ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (85.0, 25.0, -1.5) ...
* Runing NEXT_NEW sim of 1.0e+05 p

# TABLE GENERATION

In [13]:
light_table = build_table(det_name, table_type, signal_type, sensor_name, pitch)


* Getting data from NEXT_NEW - Distance: (0.0, -1.0) ...
  /Users/Javi/Development/NextLightTable/data/NEXT_NEW/dst/NEXT_NEW.x_25.0.y_25.0.z_-0.5.next.h5
  Simulation run with    100000 initial photons.
  Charge:      4 -> Sensor prob: 4.000e-05

* Getting data from NEXT_NEW - Distance: (0.0, -2.0) ...
  /Users/Javi/Development/NextLightTable/data/NEXT_NEW/dst/NEXT_NEW.x_25.0.y_25.0.z_-1.5.next.h5
  Simulation run with    100000 initial photons.
  Charge:      7 -> Sensor prob: 7.000e-05

* Getting data from NEXT_NEW - Distance: (0.0, -3.0) ...
  /Users/Javi/Development/NextLightTable/data/NEXT_NEW/dst/NEXT_NEW.x_25.0.y_25.0.z_-2.5.next.h5
  Simulation run with    100000 initial photons.
  Charge:      5 -> Sensor prob: 5.000e-05

* Getting data from NEXT_NEW - Distance: (0.0, -4.0) ...
  /Users/Javi/Development/NextLightTable/data/NEXT_NEW/dst/NEXT_NEW.x_25.0.y_25.0.z_-3.5.next.h5
  Simulation run with    100000 initial photons.
  Charge:     14 -> Sensor prob: 1.400e-04

* Getting d

In [17]:
light_table.head()

,z_m1,z_m2,z_m3,z_m4,z_m5,z_m6
dist_xy,,,,,,
0.0,0.00004,0.00007,0.00005,0.00014,0.00014,0.00017
30.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
60.0,0.00000,0.00001,0.00000,0.00000,0.00000,0.00000


In [18]:
# Storing the DataFrame
light_table_fname = table_path + get_table_fname(det_name, table_type,
                                                 signal_type, sensor_name)

if VERBOSITY:
    print(f"\n*** Storing Light Table in {light_table_fname} ...")

light_table.to_hdf(light_table_fname, '/LightTable', mode   = 'w',
                   format = 'table', data_columns = True)


*** Storing Light Table in /Users/Javi/Development/NextLightTable/data/NEXT_NEW/table/NEXT_NEW.tracking.S2.SiPM.LightTable.h5 ...
